In [ ]:
#http://scikit-learn.org/stable/modules/neighbors.html
#http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html
#https://kevinzakka.github.io/2016/07/13/k-nearest-neighbor/

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets

In [2]:
from sklearn.neighbors import KNeighborsClassifier

In [3]:
iris_aux = datasets.load_iris()
iris = pd.DataFrame(iris_aux.data, columns=iris_aux.feature_names)
iris['target'] = pd.Series(iris_aux.target)
iris['species'] = pd.Categorical.from_codes(iris_aux.target, iris_aux.target_names)
iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,species
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa


In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X = iris.iloc[:,:4]
X.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [54]:
y = iris.iloc[:,4]
y.head()

0    0
1    0
2    0
3    0
4    0
Name: target, dtype: int32

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [8]:
knn = KNeighborsClassifier(n_neighbors=3, weights='uniform', algorithm='auto', leaf_size=30, 
                           p=2, metric='minkowski', metric_params=None, n_jobs=1)
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [9]:
pred = knn.predict(X_test)

In [10]:
# evaluate accuracy
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, pred))

0.98


### CROSS VALIDATION

In [11]:
from sklearn.model_selection import GridSearchCV

### Importanto métricas de validação de CV

In [13]:
from sklearn.metrics import make_scorer, f1_score, fbeta_score, roc_auc_score, auc, roc_curve, precision_score, recall_score, classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import StratifiedKFold, KFold, LeaveOneOut

In [14]:
metrics = [f1_score,  precision_score , recall_score,  accuracy_score] #roc_auc_score]

cv_kfold =KFold(n_splits=3, shuffle=False, random_state=None)
cv_loo = LeaveOneOut()

In [49]:
#GridSearchCV(estimator, param_grid, scoring=None, fit_params=None, n_jobs=1, 
#             iid=True, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', 
#             error_score='raise', return_train_score='warn')

clf = KNeighborsClassifier()
parameters = {
    'weights' : ['uniform', 'distance'],
    'n_neighbors': [1,2,3,4,5,6] ,
    'algorithm': ['auto']
}

model = GridSearchCV(estimator = clf, 
                     param_grid = parameters, scoring= 'roc_auc', fit_params=None, n_jobs=1, iid=True, refit=True, 
                     cv=cv_kfold, verbose=0, pre_dispatch='2*n_jobs', error_score='raise', return_train_score='warn')

#scoring: forma de avaliar os parametros, o melhor pro modelo. se não passar, usa o default do modelo.
# pode receber: http://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
model

GridSearchCV(cv=KFold(n_splits=6, random_state=12, shuffle=False),
       error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'weights': ['uniform', 'distance'], 'n_neighbors': [1, 2, 3, 4, 5, 6], 'algorithm': ['auto']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [50]:
y_train.values

array([1, 2, 1, 0, 2, 1, 0, 0, 0, 1, 2, 0, 0, 0, 1, 0, 1, 2, 0, 1, 2, 0,
       2, 2, 1, 1, 2, 1, 0, 1, 2, 0, 0, 1, 1, 0, 2, 0, 0, 1, 1, 2, 1, 2,
       2, 1, 0, 0, 2, 2, 0, 0, 0, 1, 2, 0, 2, 2, 0, 1, 1, 2, 1, 2, 0, 2,
       1, 2, 1, 1, 1, 0, 1, 1, 0, 1, 2, 2, 0, 1, 2, 2, 0, 2, 0, 1, 2, 2,
       1, 2, 1, 1, 2, 2, 0, 1, 2, 0, 1, 2])

In [52]:
# X_train.values : transforma DF em matriz
#grid_fit = model.fit(X_train.values,y_train.values)
grid_fit = model.fit(X_train,pd.DataFrame(y_train, columns = ['y']))


ValueError: Found input variables with inconsistent numbers of samples: [100, 0]

In [43]:
# melhor socore:
model.best_score_

0.96

In [44]:
#melhores parametros
model.best_params_

{'algorithm': 'auto', 'n_neighbors': 4, 'weights': 'distance'}

In [48]:
pd.DataFrame(model.cv_results_)

C:\Users\gabi_\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\gabi_\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\gabi_\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\gabi_\Anaconda3\lib\site-packages\sklearn\util

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,...,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,split5_train_score,mean_train_score,std_train_score
0,0.000333,0.000471,0.001166,3.721892e-04,auto,1,uniform,"{'algorithm': 'auto', 'n_neighbors': 1, 'weigh...",1.000000,0.941176,...,0.059123,7,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
1,0.000666,0.000471,0.001000,1.205266e-06,auto,1,distance,"{'algorithm': 'auto', 'n_neighbors': 1, 'weigh...",1.000000,0.941176,...,0.059123,7,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
2,0.000169,0.000379,0.000989,1.551884e-05,auto,2,uniform,"{'algorithm': 'auto', 'n_neighbors': 2, 'weigh...",0.941176,0.941176,...,0.059123,7,0.963855,0.963855,0.987952,0.951807,0.964286,0.964286,0.966007,0.010789
3,0.000166,0.000372,0.001166,3.727222e-04,auto,2,distance,"{'algorithm': 'auto', 'n_neighbors': 2, 'weigh...",1.000000,0.941176,...,0.059123,7,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
4,0.000000,0.000000,0.000999,3.576279e-07,auto,3,uniform,"{'algorithm': 'auto', 'n_neighbors': 3, 'weigh...",0.941176,1.000000,...,0.059123,7,0.951807,0.939759,0.975904,0.963855,0.952381,0.952381,0.956015,0.011295
5,0.000167,0.000372,0.000833,3.725445e-04,auto,3,distance,"{'algorithm': 'auto', 'n_neighbors': 3, 'weigh...",1.000000,1.000000,...,0.063304,2,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
6,0.000333,0.000471,0.000333,4.710332e-04,auto,4,uniform,"{'algorithm': 'auto', 'n_neighbors': 4, 'weigh...",0.941176,1.000000,...,0.054233,2,0.951807,0.939759,0.987952,0.975904,0.940476,0.976190,0.962015,0.018841
7,0.000167,0.000372,0.000833,3.725977e-04,auto,4,distance,"{'algorithm': 'auto', 'n_neighbors': 4, 'weigh...",1.000000,1.000000,...,0.044342,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
8,0.000500,0.000500,0.000833,3.725266e-04,auto,5,uniform,"{'algorithm': 'auto', 'n_neighbors': 5, 'weigh...",1.000000,1.000000,...,0.063304,2,0.951807,0.927711,0.987952,0.951807,0.952381,0.964286,0.955991,0.017973
9,0.000333,0.000471,0.000666,4.712299e-04,auto,5,distance,"{'algorithm': 'auto', 'n_neighbors': 5, 'weigh...",1.000000,1.000000,...,0.063304,2,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
